<a href="https://colab.research.google.com/github/fbsaif007/fbsaif007/blob/main/Subcription_Breakdown_Weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
from datetime import timedelta

# Load the data
data = pd.read_csv('/content/prado_subscriptions_202411071229.csv')

# Ensure date columns are in datetime format
data['created_date_utc'] = pd.to_datetime(data['created_date_utc'], errors='coerce')
data['last_modified_date_utc'] = pd.to_datetime(data['last_modified_date_utc'], errors='coerce')
data['cancelled_date_utc'] = pd.to_datetime(data['cancelled_date_utc'], errors='coerce')

# Function to calculate weekly metrics for a given week start date and frequency type
def calculate_metrics(data_subset, week_start, week_end, frequency_label):
    active_start_week = data_subset[(data_subset['created_date_utc'] <= week_start - timedelta(days=1)) &
                                    ((data_subset['cancelled_date_utc'].isna()) |
                                     (data_subset['cancelled_date_utc'] > week_start - timedelta(days=1)))]

    new_subscribers = data_subset[(data_subset['created_date_utc'] >= week_start) &
                                  (data_subset['created_date_utc'] <= week_end)]

    cancellations = data_subset[(data_subset['cancelled_date_utc'] >= week_start) &
                                (data_subset['cancelled_date_utc'] <= week_end)]

    active_end_week = pd.concat([active_start_week, new_subscribers]).drop_duplicates(subset=['subscription_id'], keep='last')
    active_end_week = active_end_week[~active_end_week['subscription_id'].isin(cancellations['subscription_id'])]

    paused_subscribers = active_end_week[active_end_week['is_paused'] == 1]

    # Store metrics for aggregation
    metrics = {
        "Frequency": frequency_label,
        "Active Subscribers at Start of Week": len(active_start_week),
        "New Subscribers": len(new_subscribers),
        "Cancellations": len(cancellations),
        "Subscribers - EOP": len(active_end_week),
        "Paused Subscribers": len(paused_subscribers),
        "Active Subscribers": len(active_end_week) - len(paused_subscribers)
    }

    # Display the metrics for each frequency in the specified format
    print(f"\nFrequency: {frequency_label}")
    print(f"Week: {week_start.strftime('%Y-%m-%d')} to {week_end.strftime('%Y-%m-%d')}")
    print("Active Subscribers at Start of Week:", metrics["Active Subscribers at Start of Week"])
    print("New Subscribers:", metrics["New Subscribers"])
    print("Cancellations:", metrics["Cancellations"])
    print("Subscribers - EOP:", metrics["Subscribers - EOP"])
    print("Paused Subscribers:", metrics["Paused Subscribers"])
    print("Skipped (Enter manually):", "<skipped_count>")
    print("Active Subscribers:", metrics["Active Subscribers"])

    return metrics

# Main function to calculate metrics for each frequency and export to transposed Excel
def weekly_subscription_metrics():
    week_start_date = input("Enter the week start date (YYYY-MM-DD): ")
    week_start = pd.to_datetime(week_start_date)
    week_end = week_start + timedelta(days=6)

    # List to collect each frequency's metrics
    results = []

    # Calculate metrics for each frequency
    for frequency, label in zip([0, 1, 2], ['Weekly', 'Bi-weekly', 'Monthly']):
        data_subset = data[data['frequency'] == frequency]
        metrics = calculate_metrics(data_subset, week_start, week_end, label)
        results.append(metrics)

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Calculate and add totals row to the DataFrame
    totals = results_df[["Active Subscribers at Start of Week", "New Subscribers", "Cancellations",
                         "Subscribers - EOP", "Paused Subscribers", "Active Subscribers"]].sum()
    totals["Frequency"] = "Total"
    results_df = pd.concat([results_df, pd.DataFrame([totals])], ignore_index=True)

    # Display aggregated totals in the specified format
    print("\nAggregated Totals Across All Frequencies:")
    print(f"Week: {week_start.strftime('%Y-%m-%d')} to {week_end.strftime('%Y-%m-%d')}")
    print("Total Active Subscribers at Start of Week:", totals["Active Subscribers at Start of Week"])
    print("Total New Subscribers:", totals["New Subscribers"])
    print("Total Cancellations:", totals["Cancellations"])
    print("Total Subscribers - EOP:", totals["Subscribers - EOP"])
    print("Total Paused Subscribers:", totals["Paused Subscribers"])
    print("Skipped (Enter manually):", "<skipped_count>")
    print("Total Active Subscribers:", totals["Active Subscribers"])

    # Transpose the DataFrame for Excel output
    transposed_df = results_df.set_index("Frequency").T

    # Save transposed DataFrame to an Excel file
    output_path = '/content/weekly_subscription_metrics_transposed.xlsx'
    transposed_df.to_excel(output_path, index=True)

    print("\nMetrics saved to 'weekly_subscription_metrics_transposed.xlsx'")
    return transposed_df

# Run the main function
weekly_metrics_df = weekly_subscription_metrics()
weekly_metrics_df  # Display the transposed DataFrame as output


Enter the week start date (YYYY-MM-DD): 2024-10-26

Frequency: Weekly
Week: 2024-10-26 to 2024-11-01
Active Subscribers at Start of Week: 682
New Subscribers: 20
Cancellations: 16
Subscribers - EOP: 688
Paused Subscribers: 45
Skipped (Enter manually): <skipped_count>
Active Subscribers: 643

Frequency: Bi-weekly
Week: 2024-10-26 to 2024-11-01
Active Subscribers at Start of Week: 135
New Subscribers: 8
Cancellations: 5
Subscribers - EOP: 138
Paused Subscribers: 12
Skipped (Enter manually): <skipped_count>
Active Subscribers: 126

Frequency: Monthly
Week: 2024-10-26 to 2024-11-01
Active Subscribers at Start of Week: 159
New Subscribers: 16
Cancellations: 3
Subscribers - EOP: 172
Paused Subscribers: 12
Skipped (Enter manually): <skipped_count>
Active Subscribers: 160

Aggregated Totals Across All Frequencies:
Week: 2024-10-26 to 2024-11-01
Total Active Subscribers at Start of Week: 976
Total New Subscribers: 44
Total Cancellations: 24
Total Subscribers - EOP: 998
Total Paused Subscribers:

Frequency,Weekly,Bi-weekly,Monthly,Total
Active Subscribers at Start of Week,682,135,159,976
New Subscribers,20,8,16,44
Cancellations,16,5,3,24
Subscribers - EOP,688,138,172,998
Paused Subscribers,45,12,12,69
Active Subscribers,643,126,160,929
